# 发布管道

在[上一实验](labdocs/Lab06A.md)中，你创建了管道。现在要将其发布为服务。

## 连接到工作区

你首先需要使用 Azure ML SDK 连接到工作区。

> **注意**：如果 Azure 订阅的身份验证会话在你完成上一练习后已过期，系统将提示你重新进行身份验证。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 发布管道

创建并测试管道后，可以将其发布为 REST 服务。在[上一实验](labdocs/Lab06A.md)中运行过该管道，因此可以获取对该运行的引用，并用于发布管道。

In [ ]:
# 获取管道的最新运行
experiment_name = 'diabetes-training-pipeline'
pipeline_experiment = ws.experiments.get(experiment_name)
pipeline_run = list(pipeline_experiment.get_runs())[0]

# 从运行发布管道
published_pipeline = pipeline_run.publish_pipeline(
    name="Diabetes_Training_Pipeline", description="Trains diabetes model", version="1.0")

published_pipeline

请注意，发布的管道具有一个终结点，可以在 [Azure 机器学习工作室](https://ml.azure.com)的 **“终结点”** 页（**“管道终结点”** 选项卡上）中进行查看。你还可以查找其 URI，其形式为发布管道对象的属性：

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

## 使用管道终结点

要使用该终结点，客户端应用程序需要通过 HTTP 进行 REST 调用。此请求必须经过身份验证，因此需要授权标头。实际的应用程序需要使用服务主体进行身份验证，但为了测试这一点，我们将使用当前连接到 Azure 工作区的授权标头（可使用以下代码获取该标头）：

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print("Authentication header ready.")

现在可以调用 REST 接口。管道异步运行，因此我们将获得一个标识符，可用于跟踪运行中的管道试验：

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

由于你具有运行 ID，因此可以使用它来等待运行完成。

> **注意**：由于每个步骤都已配置为允许输出重用，因此管道应该会迅速完成。这主要是为了方便并节省课程时间。实际上，你可能希望每次都运行第一步，以防数据已发生更改，并且希望仅当第一步输出发生更改时才触发后续步骤。

In [ ]:
from azureml.pipeline.core.run import PipelineRun

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
pipeline_run.wait_for_completion(show_output=True)

## 计划管道

假设糖尿病患者的诊所每周收集新数据，并将其添加到数据集。可以每周运行管道以使用新数据重训练模型。

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# 每周一 00:00 UTC 提交管道
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
weekly_schedule = Schedule.create(ws, name="weekly-diabetes-training", 
                                  description="Based on time",
                                  pipeline_id=published_pipeline.id, 
                                  experiment_name=experiment_name, 
                                  recurrence=recurrence)
print('Pipeline scheduled.')

可以检索工作区中定义的计划，如下所示：

In [ ]:
schedules = Schedule.list(ws)
schedules

可以检查最新运行，如下所示：

In [ ]:
pipeline_experiment = ws.experiments.get(experiment_name)
latest_run = list(pipeline_experiment.get_runs())[0]

latest_run.get_details()

> **更多信息**：你可以在[文档](https://docs.microsoft.com/azure/machine-learning/how-to-schedule-pipelines)中查找有关计划管道的更多信息。